In [1]:
import time
import random
import asyncio
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup, ReplyKeyboardMarkup
from telegram.ext import (
    Application,
    ApplicationBuilder,
    CommandHandler,
    MessageHandler,
    ContextTypes,
    filters,
)

In [2]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
# информация о цветах: типы и стадии роста
FLOWER_TYPES = ["Цветок1", "Цветок2", "Цветок3"]
GROWTH_STAGES = ["Семечко", "Росточек", "Маленький стебелёк", "Большой стебелёк", "Цветочек"]

# кнопки для интерфейса
MAIN_MENU = [
    ["Полить 💧", "Удобрить 🌱"],  # кнопки для поливки и удобрения
    ["Инвентарь 🏷️", "Новый цветочек🌺"],  # кнопки для инвентаря и добавления цветочка
]

# частота возможности полива и удобрения
WATER_COOLDOWN = 3 * 60 * 60  # 3 часа в секундах
FERTILIZE_COOLDOWN = 12 * 60 * 60  # 12 часов в секундах

In [5]:
class Flower:
    def __init__(self, flower_type):
        self.type = flower_type # тип цветка
        self.stage = 0 # стадия роста
        self.water_count = 0 # кол-во поливов
        self.fertilize_count = 0 # кол-во удобрения
        self.last_watered = 0 # время полива
        self.last_fertilized = 0 # время удобрения
        self.notified_water = False  # флаг для уведомления о поливке
        self.notified_fertilize = False  # флаг для уведомления об удобрении

    def get_image_path(self): # получение изображений
        return f"images/{self.type.lower()}_{self.stage}.png"
    
    def can_water(self): # возможность полива
        return time.time() - self.last_watered >= WATER_COOLDOWN

    def can_fertilize(self): # возможность удобрения
        return time.time() - self.last_fertilized >= FERTILIZE_COOLDOWN

    def water(self): # полив
        if self.can_water():
            self.water_count += 1
            self.last_watered = time.time()
            self.notified_water = False  # сбрасываем флаг уведомления после действия
            return True
        return False

    def fertilize(self): # удобрение
        if self.can_fertilize():
            self.fertilize_count += 1
            self.last_fertilized = time.time()
            self.notified_fertilize = False  # Сбрасываем флаг уведомления после действия
            return True
        return False

    def time_until_water(self): # время до возможности следующего полива
        return max(0, WATER_COOLDOWN - (time.time() - self.last_watered))

    def time_until_fertilize(self): # время до возможности следующего удобрения
        return max(0, FERTILIZE_COOLDOWN - (time.time() - self.last_fertilized))

    def check_growth(self): # проверка роста
        growth_requirements = [ # требования кол-ва поливов и удобрения 
            (1, 1),  # семечко -> росточек
            (5, 2),  # росточек -> маленький стебелёк
            (9, 3),  # маленький стебелёк -> большой стебелёк
            (5, 2),  # большой стебелёк -> цветочек
        ]
        # переход на следующую стадию роста
        if self.stage < len(growth_requirements): 
            water_req, fert_req = growth_requirements[self.stage]
            if self.water_count >= water_req and self.fertilize_count >= fert_req:
                self.stage += 1
                self.water_count = 0
                self.fertilize_count = 0
                return True
        return False

In [6]:
class User:
    def __init__(self, chat_id):
        self.chat_id = chat_id # айди чата с пользователем
        self.flowers = [] # цветы конкретного пользователя
        self.add_flower() # добавить цветок

    def add_flower(self):
        new_flower = Flower(random.choice(FLOWER_TYPES)) # рандомно выбираем цветок
        self.flowers.append(new_flower) # добавляем к пользователю
        return new_flower

In [13]:
# информация о пользователях
users = {}

In [14]:
def get_user(user_id, chat_id=None):
    if user_id not in users:
        users[user_id] = User(chat_id)
    return users[user_id]

In [15]:
# обработка команды /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = get_user(update.effective_user.id, update.effective_chat.id)
    keyboard = ReplyKeyboardMarkup([["/start"]], resize_keyboard=True)
    await update.message.reply_text(
        "Добро пожаловать в твой собственный мини-сад! 🌸\n\n"
        "Для начала ты получаешь один цветочек в подарок! 🎀\n\n"
        "Используй кнопки ниже, чтобы ухаживать за своим цветочком!",
        reply_markup=keyboard,
    )
    await show_inventory(update, context)

# команда полива цветка
async def water(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = get_user(update.effective_user.id)
    current_flower = user.flowers[-1] # уточняем, что полить нужно именно последний цветок
    if current_flower.water():
        if current_flower.check_growth():
            await update.message.reply_photo( # если цветок перешел на новую стадию - показываем это пользователю
                photo=open(current_flower.get_image_path(), 'rb'),
                caption=f"Ура!🎉 Твой {current_flower.type} достиг стадии: {GROWTH_STAGES[current_flower.stage]}! 🌸",
                reply_markup=ReplyKeyboardMarkup(MAIN_MENU, resize_keyboard=True),
            )
        else:
            await update.message.reply_text(
                "Цветочек полит! 💧",
                reply_markup=ReplyKeyboardMarkup(MAIN_MENU, resize_keyboard=True),
            )
    else:
        time_left = current_flower.time_until_water() 
        hours, minutes = divmod(int(time_left // 60), 60)
        await update.message.reply_text(
            f"Нужно немного подождать, чтобы снова полить! ⏳\nПопробуй снова через {hours} часов {minutes} минут.",
            reply_markup=ReplyKeyboardMarkup(MAIN_MENU, resize_keyboard=True),
        )

async def fertilize(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = get_user(update.effective_user.id)
    current_flower = user.flowers[-1] # уточняем, что удобрить нужно именно последний цветок
    if current_flower.fertilize():
        if current_flower.check_growth():
            await update.message.reply_photo( # если цветок перешел на новую стадию - показываем это пользователю
                photo=open(current_flower.get_image_path(), 'rb'),
                caption=f"Ура!🎉 Твой {current_flower.type} достиг стадии: {GROWTH_STAGES[current_flower.stage]}! 🌸",
                reply_markup=ReplyKeyboardMarkup(MAIN_MENU, resize_keyboard=True),
            )
        else:
            await update.message.reply_text(
                "Цветочек удобрен! 🌱",
                reply_markup=ReplyKeyboardMarkup(MAIN_MENU, resize_keyboard=True),
            )
    else:
        time_left = current_flower.time_until_fertilize()
        hours, minutes = divmod(int(time_left // 60), 60)
        await update.message.reply_text(
            f"Нужно немного подождать, чтобы снова удобрить! ⏳\nПопробуй снова через {hours} часов {minutes} минут.",
            reply_markup=ReplyKeyboardMarkup(MAIN_MENU, resize_keyboard=True),
        )

# команда отображения инвентаря
async def show_inventory(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = get_user(update.effective_user.id)
    inventory = "Твои цветочки:\n\n"
    for i, flower in enumerate(user.flowers):
        inventory += f"{i + 1}. {flower.type} - {GROWTH_STAGES[flower.stage]}\n"
    await update.message.reply_text(
        inventory,
        reply_markup=ReplyKeyboardMarkup(MAIN_MENU, resize_keyboard=True),
    )
    current_flower = user.flowers[-1]
    await update.message.reply_photo( # показываем только последний цветок
        photo=open(current_flower.get_image_path(), 'rb'),
        caption=f"Текущий цветок: {current_flower.type}, Стадия: {GROWTH_STAGES[current_flower.stage]}"
    )

# добавление нового цветка
async def add_new_flower(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = get_user(update.effective_user.id)
    if user.flowers[-1].stage == len(GROWTH_STAGES) - 1:  # если текущий цветок полностью вырос
        new_flower = user.add_flower()
        await update.message.reply_text( # когда цветок вырос, даем пользователю новый
            f"Позравляем! Твой цветок полностью вырос. Ты получаешь {new_flower.type}! 🌸",
            reply_markup=ReplyKeyboardMarkup(MAIN_MENU, resize_keyboard=True),
        )
    else:
        await update.message.reply_text(
            "Сначала нужно полностью выростить этот цветочек! 🌱",
            reply_markup=ReplyKeyboardMarkup(MAIN_MENU, resize_keyboard=True),
        )

# напоминания о поливе и удобрении
async def reminders(context: ContextTypes.DEFAULT_TYPE):
    for user_id, user in users.items():
        current_flower = user.flowers[-1]
        chat_id = user.chat_id

        if current_flower.can_water() and not current_flower.notified_water:
            await context.bot.send_message(chat_id=chat_id, text="Кажется, цветочек уже можно полить!;)")
            current_flower.notified_water = True
        if current_flower.can_fertilize() and not current_flower.notified_fertilize:
            await context.bot.send_message(chat_id=chat_id, text="Кажется, цветочек уже можно удобрить!;)")
            current_flower.notified_fertilize = True

# обработка текста кнопок и связь с командами
async def handle_text(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_text = update.message.text

    if user_text == "Полить 💧":
        await water(update, context)
    elif user_text == "Удобрить 🌱":
        await fertilize(update, context)
    elif user_text == "Инвентарь 🏷️":
        await show_inventory(update, context)
    elif user_text == "Новый цветочек🌺":
        await add_new_flower(update, context)
    else:
        await update.message.reply_text("Я не понимаю эту команду. Попробуйте снова!")

In [16]:
async def main():
    # создаем приложение
    TOKEN = "your_token_here"
    application = ApplicationBuilder().token(TOKEN).build()

    # регистрируем обработчики команд
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_text))

    # постоянно проверяем наличие напоминаний
    application.job_queue.run_repeating(reminders, interval=60, first=10)

    try:
        await application.initialize()
    except Exception as e:
        print(f"Error initializing bot: {e}")

    # здесь была попытка корректно отработать завершение работы бота, но оно, к сожалению, в итоге так и не заработало корректно в Jupyter
    try:
        print("Бот запущен. Нажмите Ctrl+C для остановки.")
        await application.run_polling()
    except KeyboardInterrupt:
        print("Остановка бота...")
    finally:
        # корректное завершение
        await application.shutdown()
        print("Бот остановлен.")

In [17]:
if __name__ == "__main__":
    asyncio.run(main())

INFO:apscheduler.scheduler:Adding job tentatively -- it will be properly scheduled when the scheduler starts
DEBUG:telegram.ext.ExtBot:Calling Bot API endpoint `getMe` with parameters `{}`
DEBUG:httpcore.connection:connect_tcp.started host='api.telegram.org' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000001BC39B302F0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x000001BC3C0A7AD0> server_hostname='api.telegram.org' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000001BC39B30C20>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:

Бот запущен. Нажмите Ctrl+C для остановки.


DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000001BC3C09F620>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x000001BC3C0A7A50> server_hostname='api.telegram.org' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000001BC3C117CE0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:apscheduler.scheduler:Looking for jobs to run
DEBUG:apscheduler.scheduler:Next wakeup is due at 2025-01-16 19:26:03.255858+00:00 (in 59.988993 seconds)
INFO:apscheduler.executors.default:Running job "reminders (trigger: interval[0:01:00], next run at: 

Бот остановлен.


RuntimeError: Cannot close a running event loop